In [2]:
using Distributed

In [3]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [4]:
@everywhere using DistributedArrays, DistributedJets, Jets, WaveFD, JetPackWaveFD, DistributedOperations, Schedulers

┌ Info: Precompiling Schedulers [fcb5363a-df8d-57f1-a64b-7615e60d793b]
└ @ Base loading.jl:1278


In [5]:
v = read!("../02_marmousi_model_setup/marmousi_vp_20m_176x851.bin", Array{Float32}(undef, 176,851));
dz,dx = 20.0,20.0
nz,nx = size(v)
@show dz,dx
@show nz,nx;

(dz, dx) = (20.0, 20.0)
(nz, nx) = (176, 851)


In [6]:
sx = range(0,length=4,stop=(851-1)*20)
nshots = length(sx)
@show nshots;

nshots = 4


## Broadcast the velocity (v) to workers

In [7]:
_v = bcast(v)

ArrayFutures with pids=[2, 3, 1] and type (176, 851)

## Build a list of keyword arguments to be passed to the modeling operator

In [8]:
kwargs = (reportinterval = 0, freqQ=5, srcfieldfile="", nsponge = 200)

(reportinterval = 0, freqQ = 5, srcfieldfile = "", nsponge = 200)

## Define function to be run by the Scheduler

In [11]:
@everywhere function modelshot(isrc,sx,_v; kwargs...)
    nz,nx,dz,dx = 176,851,20.0,20.0   
    @info "modeling shot $(isrc) on $(gethostname()) with id $(myid())..."
    F = JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        isinterior = true,
        ntrec = 1101,
        dtrec = 0.002,
        dtmod = 0.001,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=5.0),
        sx = sx[isrc],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        kwargs...)
    
    d = F*_v
    @info "...done modeling shot $(isrc) on $(gethostname()) with id $(myid())"
    @info "extrema of shot $(isrc) is $(extrema(d))"

    # typically write to cloud storage here
    nothing
end

## Use cvxpmap to schedule tasks to workers (dynamic schedule)

In [13]:
epmap(i->modelshot(i,sx,localpart(_v);kwargs...), 1:nshots)

┌ Info: running task 1 on process 3; 2 workers total; 3 tasks left in task-pool.
└ @ Schedulers /home/cvx/.julia/dev/Schedulers/src/Schedulers.jl:159
┌ Info: running task 2 on process 2; 2 workers total; 2 tasks left in task-pool.
└ @ Schedulers /home/cvx/.julia/dev/Schedulers/src/Schedulers.jl:159
[ Info: modeling shot 2 on cbox-wask-D16s-v3 with id 2...
[ Info: modeling shot 1 on cbox-wask-D16s-v3 with id 3...
[ Info: ...done modeling shot 2 on cbox-wask-D16s-v3 with id 2
[ Info: ...done modeling shot 1 on cbox-wask-D16s-v3 with id 3
[ Info: extrema of shot 2 is (-58.7598f0, 107.51882f0)
┌ Info: running task 3 on process 2; 2 workers total; 1 tasks left in task-pool.
└ @ Schedulers /home/cvx/.julia/dev/Schedulers/src/Schedulers.jl:159
[ Info: modeling shot 3 on cbox-wask-D16s-v3 with id 2...
[ Info: extrema of shot 1 is (-59.97956f0, 109.98762f0)
┌ Info: running task 4 on process 3; 2 workers total; 0 tasks left in task-pool.
└ @ Schedulers /home/cvx/.julia/dev/Schedulers/src/Schedul

## Remove workers

In [14]:
rmprocs(workers())

Task (done) @0x00007fbe1b3e2230